# MIE562 Project - Truck Scheduling - MIP Model

**Team 5 - Dylan Camus, Ryan Do, Fan Jia, Matheus Magalhaes, Sugumar Prabhakaran**

**Date: 8 Nov 2020**

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

import gurobipy as gp
from gurobipy import Model, GRB, quicksum, max_
from random import randint, seed

seed(0)

## Introduction

### 1. Sets
       
   * $j \in J$: A carrier, $j$, in a set of carriers, $J$: 
       * Ex. $J = \left \{1,2,3,\ldots,j  \right \}$
       
   * $|J|$ = n: number of carriers
       
   * $k_{j}$: number of containers for each carrier $j$:    
   
   * $k \in K$: A container, $k$ in a set of containers, $K$: 
       * Ex. $K = \left \{1,2,3... \right \}$   

   * $c \in C$: A chassis, $c$, in a set of chassis, $C$: 
       * Ex. $C = \left \{1,2,3,\ldots,c  \right \}$
 
   * $L = \left \{1,2,3\right\}$: is the set of travel legs, where:
       * $1$ : terminal to transloading facility leg
       * $2$ : terminal to stack leg
       * $3$ : stack to transloading facility leg

### 2. Parameters

   * $\Phi_{kl}$  : Time duration transporting container k on leg l, which includes a variable processing time for legs 1 and 3 [days] 
   * $D_{ll'}$: Delay required for a chassis to start leg l' after completing l (including travel) [days]
   * $M$     : Some large number
   * $R_k$   : Release date for container $k$ [days]
   
   * $T_j^{''}$: free period before demurrage cost for carrier j at terminal [days]
   
   * $T_j$: demurrage cost/unit time for carrier j at terminal [($/days)/container]
   
   * $S$     : fixed cost at stack [$/container]
   
   * ${S}'$  : variable cost at stack [($/day)/container]
   
   * $\delta_j^{''}$: free period before detention cost for containers of carrier j [days]
   
   * $\delta_j$: detention cost/unit time for containers of carrier j [($/days)/container]
   
   * $\rho_k$ : priority factor of container k (higher value is higher priority)

   * $Y_{jk}$ : binary variable = 1 if container $k$ belongs to carrier $j$
   

In [77]:
# read and construct parameters from standard instance interface
instance_csv_path = "test_instance.csv"

df = pd.read_csv(instance_csv_path)
print(df)

nK, nJ, nC, S, S_prime = tuple(df.iloc[0].dropna().astype(int))
C = list(range(nC))
K = [k for k in df.iloc[1].dropna().astype(int)]
assert nK == len(K)
J = list(set(df.iloc[2].dropna()))
assert nJ == len(J)

Y = {j:{k: 0 for k in K} for j in J}
for i, k in enumerate(K):
    Y[df.iloc[2].dropna()[i]][k] = 1
    
R = {k: int(df.iloc[3].dropna()[i]) for i, k in enumerate(K)}
p = {k: int(df.iloc[4].dropna()[i]) for i, k in enumerate(K)}
rho = {k: int(df.iloc[5].dropna()[i]) for i, k in enumerate(K)}

T_prime = {df.iloc[6].dropna()[i]: int(df.iloc[6].dropna()[i+1]) for i in range(0, len(df.iloc[6].dropna()), 2)}
T = {df.iloc[7].dropna()[i]: int(df.iloc[7].dropna()[i+1]) for i in range(0, len(df.iloc[7].dropna()), 2)}
Delta_prime = {df.iloc[8].dropna()[i]: int(df.iloc[8].dropna()[i+1]) for i in range(0, len(df.iloc[8].dropna()), 2)}
Delta = {df.iloc[9].dropna()[i]: int(df.iloc[9].dropna()[i+1]) for i in range(0, len(df.iloc[9].dropna()), 2)}
S, S_prime = tuple(df.iloc[10].dropna().astype(int))

L = [1, 2, 3]
leg_times = {l: float(df.iloc[11][i*2+1]) for i, l in enumerate(L)}
Phi = {k: {1: 2*leg_times[1] + p[k], 2: leg_times[2], 3: leg_times[3] + p[k] + leg_times[1]} for k in K} # time duration of leg l (including processing time) in hours #
D = {1:{1: 0, 2: 0, 3: leg_times[2]}, 
     2:{1: leg_times[2], 2: leg_times[2], 3: 0}, 
     3:{1: 0, 2: 0, 3: leg_times[2]}} 

M = 1000   

   test_instance Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0             10          3          5        300         15        NaN   
1              1          2          3          4          5          6   
2              a          a          a          b          b          b   
3              0          0          0          0          0          4   
4             10         10         20         30         40          1   
5              1          2          3          4          5          6   
6              a          3          b          2          c          5   
7              a       1000          b       1400          c       1800   
8              a         21          b         14          c         18   
9              a         50          b         75          c        100   
10           300         15        NaN        NaN        NaN        NaN   
11         leg_1      0.125      leg_2      0.125      leg_3      0.125   

   Unnamed: 6 Unnamed: 7

In [3]:
# create a list of tuples containing every iteration of k,l,c
A = [(k,l,c) for k in K for l in L for c in C]

In [4]:
#Create a new model
model = gp.Model("Truck_Scheduling")
model.setParam('TimeLimit', 60) # seconds

Using license file C:\Users\doryan\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


### 3. Decision Variables

   * Binary Variable: $x_{klc}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if}\ \text{container k travels leg l on chassis c} \\
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $
   * Start time $s$ of container $k$ on leg $l$ on chassis $c$: $s_{klc}$, where  $s_{klc} \geq 0$
   * $z_{klk^{'}l^{'}c} = 1$ if the transport job of container $k^{'}$ on chassis c along leg $l^{'}$ is scheduled after the transport job of container $k$ on chassis $c$ along leg $l$.


In [5]:
#Create decision variables
x = model.addVars(A, vtype=GRB.BINARY, name="x_klc")
s = model.addVars(A, vtype=GRB.CONTINUOUS, name="s_klc")
z = model.addVars(K, L, K, L, C, vtype=GRB.BINARY, name="z_klk'l'c")

#Max variables
max_expression_resource_constraint_1 = model.addVars(K, L, K, L, C, vtype=GRB.CONTINUOUS, name="max resource a")
max_expression_resource_constraint_2 = model.addVars(K, L, K, L, C, vtype=GRB.CONTINUOUS, name="max resource b")
max_expression_demurrage = model.addVars(K, L, C, J, vtype=GRB.CONTINUOUS, name="max_expression_demurrage")
max_expression_detention = model.addVars(K, L, C, J, vtype=GRB.CONTINUOUS, name="max_expression_detention")


### 4. Objective Function

We want to minimize the Total Cost ($c$) = (Demurrage cost}) + (Detention cost) + (Stack cost) + (Priority penalty cost)
   
   * **objective function:** $min$ $(c)$
   
   * $c = c_{dem} + c_{det} + c_{stk} + c_{pri}$

**(1) Demurrage Cost:**

The Demurrage Cost ($c_{dem}$) refers to cost associated with keeping containers in the terminal beyond the free period allotedin days ($T_{j}^{''}$) that is different for each carrier.

If the start time for a container ($s_{klc}$) is higher than the release date ($R_{k}$) + the free period ($T_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.
   
   * $c_{dem} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,2\}}\sum_{j \in J}T_jx_{klc}Y_{jk}\cdot max\left [(s_{klc} - R_k - T_{j}^{''}), 0  \right ]$
   
**(2) Detention Cost**

The terminal also charges a detention cost ($c_{det}$) for containers that are not returned to the terminal for processing before return to the carrier.  Similar to demurrage, there is a free period before which the containers must be returned to the terminal ($\delta_{j}^{''}$).

If the start time for a container ($s_{klc}$) and the total travel time of a container back to the terminal including processing time ($\phi_{kl}$) is greater than the release date ($R_{k}$) + the free period ($\delta_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,3\}}\sum_{j \in J}\delta_jx_{klc}Y_{jk}\cdot max\left [(s_{klc}+ \phi_{kl} - (R_k+\delta_{j}^{''})), 0  \right ]$

**(3) Stack Storage Cost**

The stack cost ($c_{stk}$) is the sum of the fixed cost percontainer ($S$) and the variable cost per day per container ($S^{'}$) x the number of days that a container is in the stack, which is the difference between leg 3 and leg 2 start times ($s_{klc}$).

   * $c_{stk} = \sum_{k \in K}\sum_{c \in C}x_{k2c}\cdot (S + S'(s_{k3c} - (s_{k2c} + \phi_{k2})))$

**(4) Priority Penalty Cost**

The priority penalty cost ($c_{pri}$) is applied for every day that a container sits in the terminal beyond the release day ($R_{k}$).  Since the priority factor per container ($\rho_{k}$) is higher for higher priority containers, the associated penalty costs will be higher.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{1,2\}}\rho_k\cdot(s_{klc} - R_k)\cdot x_{klc}$

In [6]:
# demurrage cost days max
model.addConstrs(max_expression_demurrage[k,l,c,j] >= (s[k,l,c] - R[k] - T_prime[j]) for k in K for c in C for l in [1, 2] for j in J)
model.addConstrs(max_expression_demurrage[k,l,c,j] >= 0 for k in K for c in C for l in [1, 2] for j in J)

# detention cost days max
model.addConstrs(max_expression_detention[k,l,c,j] >= (s[k,l,c] + Phi[k][l] - (R[k] + Delta_prime[j])) for k in K for c in C for l in [1, 3] for j in J)
model.addConstrs(max_expression_detention[k,l,c,j] >= 0 for k in K for c in C for l in [1, 3] for j in J)

#Define objective function
demurrage_cost = quicksum(x[k,l,c]*T[j]*Y[j][k]*max_expression_demurrage[k,l,c,j] 
                          for k in K for c in C for l in [1, 2] for j in J) 
detention_cost = quicksum(x[k,l,c]*Delta[j]*Y[j][k]*max_expression_detention[k,l,c,j] 
                          for k in K for c in C for l in [1, 3] for j in J)
stack_cost = quicksum(x[k,2,c]*(S + S_prime*(s[k,3,c] - (s[k,2,c] + Phi[k][2]))) for k in K for c in C)

priority_cost = quicksum(x[k,l,c]*rho[k]*(s[k,l,c] - R[k]) for k in K for c in C for l in [1, 2])

model.setObjective(demurrage_cost 
                   + detention_cost 
                   + stack_cost 
                   + priority_cost 
                   , GRB.MINIMIZE)
# model.setObjective(0, GRB.MINIMIZE) demurrage_cost + detention_cost + stack_cost + priority_cost

### 5. Constraints

**(1) Resource constraints**

For each chassis, transport jobs can't overlap and chassis (resources) may have to travel between jobs. 

For the case where job $klc$ scheduled after job $k'l'c$, i.e. $s_{klc} > s_{k'l'c}$:

   * $M(1-x_{klc}) + M(1-x_{k'l'c}) + (s_{klc} - s_{k'l'c}) \geq \phi_{k'l'} + D_{l'l} - M\cdot z_{klk'l'c}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k,k' \in K, l,l'\in L, c \in C$ &nbsp;$s.t.$&nbsp; $(k=k' \neg\wedge l=l') \wedge k\leq k'$
   
For the case where job $k'l'c$ scheduled after job $klc$, i.e. $s_{klc} > s_{k'l'c}$:

   * $M(1-x_{klc}) + M(1-x_{k'l'c}) + (s_{k'l'c} - s_{klc}) \geq \phi_{kl} + D_{ll'} - M\cdot(1-z_{klk'l'c})$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k,k' \in K, l,l'\in L, c \in C$ &nbsp;$s.t.$&nbsp; $(k=k' \neg\wedge l=l') \wedge k\leq k'$

In [7]:
# max constraints (necessary to keep max variable geq than arguments)
# model.addConstrs(max_expression_resource_constraint_1[k,l,kp,lp,c] >= M * (1 - x[k,l,c]) 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_1[k,l,kp,lp,c] >= M * (1 - x[kp,lp,c]) 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_1[k,l,kp,lp,c] >= s[k,l,c] - s[kp,lp,c]
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))

# model.addConstrs(max_expression_resource_constraint_2[k,l,kp,lp,c] >= M * (1 - x[k,l,c]) 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_2[k,l,kp,lp,c] >= M * (1 - x[kp,lp,c]) 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_2[k,l,kp,lp,c] >= s[kp,lp,c] - s[k,l,c]
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))

# constrain max variable (necessary to keep max variable from being freely above s_klc - s_kplpc)
# model.addConstrs(max_expression_resource_constraint_1[k,l,kp,lp,c] 
#                  == M*(1 - x[k,l,c]) + M*(1 - x[kp,lp,c]) + (s[k,l,c] - s[kp,lp,c])
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_2[k,l,kp,lp,c] 
#                  == M*(1 - x[k,l,c]) + M*(1 - x[kp,lp,c]) + (s[kp,lp,c] - s[k,l,c])
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))

# model.addConstrs(max_expression_resource_constraint_1[k,l,kp,lp,c] >= Phi[kp][lp] + D[lp][l] - M * z[k, l, kp, lp, c] 
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
# model.addConstrs(max_expression_resource_constraint_2[k,l,kp,lp,c] >= Phi[k][l] + D[l][lp] - M * (1-z[k, l, kp, lp, c])
#                  for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))


model.addConstrs(M*(1 - x[k,l,c]) + M*(1 - x[kp,lp,c]) + (s[k,l,c] - s[kp,lp,c]) >= Phi[kp][lp] + D[lp][l] - M * z[k, l, kp, lp, c] 
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))
model.addConstrs(M*(1 - x[k,l,c]) + M*(1 - x[kp,lp,c]) + (s[kp,lp,c] - s[k,l,c]) >= Phi[k][l] + D[l][lp] - M * (1-z[k, l, kp, lp, c])
                 for k in K for l in L for kp in K for lp in L for c in C if (not(k == kp and l == lp) and (k <= kp)))



{(1, 1, 1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2, 2, 1): <gurobi.Constr *Awaiting Model Update*

**(2) Release date constraint**

Start time ($s_{klc}$) must be after release date ($R_k$) for every container that leaves terminal on either leg 1 or leg 2 (ie. $x_{klc}=1$):

   * $s_{klc} \geq R_{k}x_{klc}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$, $l\in L$, $c \in C$ 

In [8]:
model.addConstrs(s[k,l,c]>= R[k]*x[k,l,c] for k in K for l in L for c in C)

{(1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Constr *Awaiting Model Upd

**(3) Containers going through leg 2 must go through leg 3**

Each container k will have a x_klc value of 1 or 0 for both leg 2 and leg 3 depending on if they travel through them or not:

   * $\sum_{c \in C}(x_{k2c}) = \sum_{c \in C}(x_{k3c})$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$

In [9]:
model.addConstrs(quicksum(x[k,2,c] for c in C)==quicksum(x[k,3,c] for c in C) for k in K)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

**(4) Container must go through leg 1 OR (leg 2 AND 3)**

Each container ($k$) will have $x_klc = 1$ for either leg 1 or leg 2 so the sum of the two must be 1 for all containers ($k$) on all chassis ($c$).

   * $\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(x_{k2c}) = 1$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$

In [10]:
model.addConstrs(
    (quicksum(x[k,1,c] for c in C) + quicksum(x[k,2,c] for c in C))==1 for k in K
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

**(5) Precedence Constraint: Leg 2 must be before Leg 3**

Each container k will have a $x_{klc}$ value of 1 for either leg 1 or leg 2 so the sum of the two must be 1 for all containers (k) on all chassis (c).

   * $M\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(s_{k3c}) \geq = \sum_{c \in C}(s_{k2c}) + \Phi_{k2}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$
   

In [11]:
model.addConstrs(
    M*(quicksum(x[k,1,c] for c in C)) + quicksum(s[k,3,c] for c in C) >= 
    (quicksum(s[k,2,c] for c in C) + Phi[k][2]) for k in K
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

**(6) Bind x=0 to s=0**

When $x_{klc} = 0, s_{klc} = 0 $

   * $Mx_{klc} \geq s_{klc}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$, $l \in L$, $c \in C$

In [12]:
model.addConstrs(M*x[k,l,c] >= s[k,l,c] for k in K for l in L for c in C)

{(1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Constr *Awaiting Model Upd

**(7) Domain of $s_{klc}$**

The start time ($s_{klc}$) must be greater than 0:

   * $s_{klc} \geq 0$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$, $l \in L$, $c \in C$

In [13]:
model.addConstrs(s[k,l,c] >=0 for k in K for l in L for c in C)

{(1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Constr *Awaiting Model Upd

In [14]:
model.optimize()

for k in K:
    for l in L:
        for c in C:
            if (x[k,l,c].X != 0):
                print(x[k,l,c], s[k,l,c])
#             for kp in K:
#                 for lp in L:
#                     print(x[k,l,c], s[k,l,c], x[kp, lp, c], s[kp, lp, c], z[k, l, kp, lp, c])

print(demurrage_cost.getValue(),
detention_cost.getValue(),
stack_cost.getValue(),
priority_cost.getValue())


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 6330 rows, 14700 columns and 26075 nonzeros
Model fingerprint: 0xb6ce0be1
Model has 350 quadratic objective terms
Variable types: 10050 continuous, 4650 integer (4650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+02]
  QObjective range [2e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 3e+03]
Found heuristic solution: objective 2.950000e+13
Presolve removed 1225 rows and 11875 columns
Presolve time: 0.05s
Presolved: 5705 rows, 3575 columns, 50402 nonzeros
Presolved model has 400 SOS constraint(s)
Variable types: 900 continuous, 2675 integer (2675 binary)

Root relaxation: objective -9.217088e+04, 1307 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -92170.875    0   56 2.9500e+13 -92170.875   100%

In [76]:
# Visualize optimized schedule
from datetime import datetime, timedelta
import plotly.graph_objects as go

raw_df = []
start_date = datetime(2020, 11, 1, 0, 0)

for k in K:
    for l in L:
        for c in C:
            if (x[k,l,c].X != 0):
                raw_df.append(dict(Chassis=str(c), 
                                   Start=start_date + timedelta(days=int(s[k,l,c].X), 
                                                                hours=(s[k,l,c].X % 1 * 24)),
                                   Finish=start_date + timedelta(days=int(s[k,l,c].X + Phi[k][l]), 
                                                                 hours=((s[k,l,c].X + Phi[k][l]) % 1 * 24)),
                                   Leg=str(l), 
                                   Container=str(k))
                             )
                
df = pd.DataFrame(raw_df)
fig_chassis = px.timeline(df, x_start="Start", x_end="Finish", y="Chassis", color="Leg")
fig_chassis.show()


fig_container = px.timeline(df, x_start="Start", x_end="Finish", y="Container", color="Chassis")

for k in K:
    fig_container.add_trace(
        go.Scatter(
            x = [start_date + timedelta(days=int(R[k]), hours=(R[k] % 1 * 24)),
                 start_date + timedelta(days=int(R[k]), hours=(R[k] % 1 * 24))],
            y = [k-0.4, k+0.4],
            mode = "lines",
            line = go.scatter.Line(color = "black", width = 1),
            showlegend = False
        )
    )
fig_container.show()

fig_chassis.write_html(f"{instance_csv_path}_schedule_.html")
fig_container.write_html("")
                


# 